## Story Cloze: Bert for Sequence Classification

#### install tools

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00


In [2]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import datasets
from datasets import load_dataset, load_metric
from datasets import Dataset
from sklearn.metrics import recall_score


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# load dataset
dataset = pd.read_json('/content/drive/MyDrive/Project/datasets/ROCStories_winter2017_mixed.json')

### Modify Training Test and Validation Datasets

In [6]:
def modify_training(data):
  new = pd.DataFrame()
  new['stories'] = data.story
  new['ending0'] = data.ending0
  new['ending1'] = data.ending1
  new['labels'] = data.label
  ntest = new.melt(id_vars=['stories','labels'], value_vars=['ending0', 'ending1'], var_name='ending_number', value_name='endings')
  check_last_index_match = lambda row: 1 if row['ending_number'][-1] == str(row['labels']) else 0
  ntest['label']=ntest.apply(check_last_index_match, axis=1)
  ntest.drop(['labels','ending_number'],inplace=True,axis=1)
  return ntest.sample(frac=1,replace=False).reset_index(drop=True)

In [7]:
data_mod = modify_training(dataset)
train_df = data_mod[:70000]
validation_df = data_mod[70000:85000]
test_df = data_mod[85000:]

In [8]:
print('Maximum Num Training Words:', max([len(x.split()) for x in train_df.stories]) + max([len(x.split()) for x in train_df.endings]) )

Maximum Num Training Words: 78


In [9]:
### turn all train, validation, and test datasets into a dataset dictionary ###
train_dataset = Dataset.from_dict(train_df)
val_dataset = Dataset.from_dict(validation_df)
test_dataset = Dataset.from_dict(test_df)
data_dict = datasets.DatasetDict({"train":train_dataset,
                                        "validation" : val_dataset,
                                        "test":test_dataset})

## Look at some examples

In [10]:
def show_one(example):
    print(f"Story: {example['stories']}")
    print(f"\nEnding: {example['endings']}")
    print(f"\nLabel: {example['label']}")

In [11]:
show_one(data_dict['train'][999])

Story: Roy was the fastest runner on the team. His team made it to the state finals. Roy decided to get extra practice to prepare for the state finals. He ran an extra hour everyday.

Ending: Her mom worked with the parks department to get the swings replaced.

Label: 0


In [12]:
show_one(data_dict['validation'][999])

Story: Sam went on a date with a girl he met at a bar. He didn't remember much about her. The date went horribly. Sam made up an excuse to leave early.

Ending: And that weekend, he got to go to the concert!

Label: 0


## Preprocess

In [13]:
### load tokenizer ###
from transformers import AutoTokenizer
checkpoint = '/content/drive/MyDrive/Project/bert_model_weights'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#### Tokenize each sentence

In [14]:
MAX_SEQUENCE_LENGTH = 85
def preprocess_function(examples):
    return tokenizer(examples['stories'], examples['endings'], max_length=MAX_SEQUENCE_LENGTH,
                     truncation=True, padding='max_length',return_tensors='tf')

In [15]:
pre_tokenizer_columns = set(data_dict["train"].features)
encoded_dataset = data_dict.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20330 [00:00<?, ? examples/s]

Columns added by tokenizer: ['token_type_ids', 'input_ids', 'attention_mask']


#### Look at some tokenization and preprocessing outputs

In [16]:
examples = data_dict["train"][:3]
features = preprocess_function(examples)
print(
    len(features["input_ids"]),
    len(features["input_ids"][0]),
    [len(x) for x in features["input_ids"]],
)

3 85 [85, 85, 85]


In [17]:
[tokenizer.decode(features["input_ids"][x]) for x in range(len(features))]

['[CLS] tom suffered from night terrors. it made it hard to have women spend the night. he told his current girlfriend about it. she was very understanding and knew a lot about it. [SEP] so she did not go swimming after all. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 "[CLS] bob worked in marketing marketing department of a company. bob decided to have a get together with his coworkers in a hotel room. bob ordered 8 extra large pizzas with the works. bob ended up eating most of the pizza and enraging his coworkers. [SEP] the dog's owners were so grateful to anna! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]",
 '[CLS] sam has a biology test in the morning. he was not ready for it all. he decided to stay up and study. he ended up sleeping through his ala

## Load Model

In [18]:
from transformers import TFBertForSequenceClassification

id2label = {0: "Invalid", 1: "Valid"}
label2id = {val: key for key, val in id2label.items()}

model = TFBertForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model.config.dropout = 0.3
model.config.hidden_dropout_prob = 0.3
model.config.attention_probs_dropout_prob = 0.3

In [20]:
tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset["train"],
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    encoded_dataset["validation"],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)

tf_test_dataset = model.prepare_tf_dataset(
    encoded_dataset["test"],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)

Evaluation Metrics

In [21]:
import evaluate
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")

In [22]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    acc = accuracy.compute(predictions=predictions, references=labels)['accuracy']
    rec = recall.compute(predictions=predictions, references=labels)['recall']

    return {"accuracy": acc, "recall": rec}

In [23]:
from transformers import create_optimizer

num_epochs = 3
batch_size = 16
batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=3e-05, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

In [24]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset
)

In [25]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=[metric_callback],
)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4375/4375 [==============================] - 1555s 337ms/step - loss: 0.1099 - val_loss: 0.0941 - accuracy: 0.9619 - recall: 0.9761
Epoch 2/3
4375/4375 [==============================] - 1488s 340ms/step - loss: 0.0326 - val_loss: 0.0948 - accuracy: 0.9672 - recall: 0.9657
Epoch 3/3
4375/4375 [==============================] - 1488s 340ms/step - loss: 0.0062 - val_loss: 0.1399 - accuracy: 0.9685 - recall: 0.9683


In [27]:
model.evaluate(tf_test_dataset)

1271/1271 [==============================] - 132s 104ms/step - loss: 0.1346


0.13460350036621094

In [33]:
predictions = model.predict(tf_test_dataset)

1271/1271 [==============================] - 150s 105ms/step


In [36]:
classifications = np.argmax(predictions.logits, axis=1)

In [46]:
print('Test Accuracy:', np.mean(classifications == encoded_dataset["test"]['label']))

Test Accuracy: 0.9696015740285293


In [45]:
print("Test Recall:" , recall_score(encoded_dataset["test"]['label'], classifications))

Test Recall: 0.9710560110639138
